<a href="https://colab.research.google.com/github/mahesh-keswani/ML-DL-Basics/blob/main/keras_examples/CustomLayersTF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist 

In [2]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [3]:
x_train = x_train.reshape(-1, 28*28).astype("float32") / 255.0
x_test = x_test.reshape(-1, 28*28).astype("float32") / 255.0

In [4]:
class MyModel(keras.Model):
    def __init__(self, num_classes):
        super(MyModel, self).__init__()
        self.dense1 = layers.Dense(64, activation='relu')
        self.dense2 = layers.Dense(num_classes)
    
    def call(self, input_tensor):
        x = self.dense1(input_tensor)
        x = self.dense2(x)
        return x

In [5]:
model = MyModel(num_classes=10)

model.compile( loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'], optimizer=keras.optimizers.Adam() )

In [6]:
model.fit(x_train, y_train, batch_size=32, verbose=2, epochs=3)

Epoch 1/3
1875/1875 - 2s - loss: 0.3097 - accuracy: 0.9131
Epoch 2/3
1875/1875 - 2s - loss: 0.1544 - accuracy: 0.9552
Epoch 3/3
1875/1875 - 2s - loss: 0.1139 - accuracy: 0.9668


In [7]:
model.evaluate(x_test, y_test, batch_size=32, verbose=2)

313/313 - 0s - loss: 0.1117 - accuracy: 0.9658


[0.11173871904611588, 0.9657999873161316]

<h1>Now creating custom layer :D</h1>

In [8]:
# Different initializations: https://www.tensorflow.org/api_docs/python/tf/keras/initializers

class MyDense(layers.Layer):
    def __init__(self, units, input_dim):
        super(MyDense, self).__init__()
        self.w = self.add_weight( name='w', shape=(input_dim, units), initializer='random_normal', trainable=True)
        self.b = self.add_weight( name='b', shape=(units, ), initializer='zeros', trainable=True)

    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b

In [9]:
class MyModel(keras.Model):
    def __init__(self, num_classes, input_dim):
        super(MyModel, self).__init__()
        self.dense1 = MyDense(units=64, input_dim=input_dim)
        self.dense2 = MyDense(units=10, input_dim=64)
    
    def call(self, input_tensor):
        x = tf.nn.relu( self.dense1(input_tensor) )
        x = self.dense2(x)
        return x

In [10]:
model = MyModel(input_dim=784, num_classes=10)

model.compile( loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'], optimizer=keras.optimizers.Adam() )

In [11]:
model.fit(x_train, y_train, batch_size=32, verbose=2, epochs=3)

Epoch 1/3
1875/1875 - 2s - loss: 0.3412 - accuracy: 0.9061
Epoch 2/3
1875/1875 - 2s - loss: 0.1623 - accuracy: 0.9534
Epoch 3/3
1875/1875 - 2s - loss: 0.1168 - accuracy: 0.9650


In [12]:
model.evaluate(x_test, y_test, batch_size=32, verbose=2)

313/313 - 0s - loss: 0.1083 - accuracy: 0.9679


[0.10832881182432175, 0.9678999781608582]